# Read in df and keyword list

In [1]:
# import packages
import os
import pandas as pd
import nltk
from nltk import WhitespaceTokenizer
from nltk.corpus import stopwords
from string import punctuation as original_punct
import re
from collections import Counter
import string
from nltk.stem import WordNetLemmatizer 

# read in data
df = pd.read_csv("Data_IRS_ExcludeMuseumOnly.csv")

# read in list of keywords
keyword_df = pd.read_csv("Search Terms_TT.csv")


### Clean names

In [4]:
# get name column and make lowercase
names = df["name"]
names = [str(name).lower() for name in names]

# remove word equivalents
names2 = []
for name in names:
    name = re.sub(r"\bctr\b", "center", name)
    name = re.sub(r"\bcntr\b", "center", name)
    name = re.sub(r"\bassn\b", "association", name)
    name = re.sub(r"\bassoc\b", " association", name)
    names2.append(name)
    
# create stop word list
nltk.download('stopwords')
stop_words = stopwords.words('english') + list(original_punct)

# define lemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer() 

name_tokens = []
for name in names2:
    name = WhitespaceTokenizer().tokenize(name)           # tokenize
    name = filter(lambda x: x.isalpha(), name)            # remove numerics
    name = [lemmatizer.lemmatize(w) for w in name]        # lemmatize
    name = [w for w in name if not w in stop_words]       # remove stop words 
    name_tokens.append(name)
    
# string tweets together again
names = [" ".join(name) for name in name_tokens] 

df["name"] = names

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ydeng\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ydeng\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Clean keywords

In [5]:
# make keywords lowercase
keywords = keyword_df["Keyword"]
keywords = [str(keyword).lower() for keyword in keywords]


# lemmatize keywords
keyword_tokens = []
for keyword in keywords:
    keyword = WhitespaceTokenizer().tokenize(keyword)           # tokenize
    keyword = [lemmatizer.lemmatize(w) for w in keyword]        # lemmatize
    keyword = [w for w in keyword if not w in stop_words]       # remove stop words 
    keyword_tokens.append(keyword)
    
# string keywords together again and put back in keyword_df
keywords = [" ".join(keyword) for keyword in keyword_tokens]
keyword_df["Keyword"] = keywords

keyword_df.drop_duplicates("Keyword")
keywords = list(set(keywords))

In [83]:
# build new abbreviated dataframe with only rows where there is a NA ctee_cd or a keyword match

# df with missing ctee_cd
df_na = df[df["ntee_cd"].isna()] 
df_na["keyword"] = ""

# df with keyword matches from list 1
is_1 = keyword_df["Include"] == 1
keyword_df_1 = keyword_df[is_1]
df_keyword_1 = pd.DataFrame()
for keyword in keyword_df_1["Keyword"]:
    chunk = df[df['name'].str.contains(keyword)]
    chunk["keyword"] = keyword
    df_keyword_1 = pd.concat([df_keyword_1, chunk], ignore_index=True)  # add rows that contain this keyword

# add "-seum" matches to df_keyword_1
chunk = df[df['name'].str.contains("seum")]  # any containing "museum" will be removed during drop_duplicates later on
chunk["keyword"] = "seum"
df_keyword_1 = pd.concat([df_keyword_1, chunk], ignore_index=True)   
df_keyword_1 = df_keyword_1.drop_duplicates()
    
# df with keyword matches from list 2
is_2 = keyword_df["Include"] == 2
keyword_df_2 = keyword_df[is_2]
df_keyword_2 = pd.DataFrame()
for keyword in keyword_df_2["Keyword"]:
    chunk = df[df['name'].str.contains(keyword)]
    chunk["keyword"] = keyword
    df_keyword_2 = pd.concat([df_keyword_2, chunk], ignore_index=True)  # add rows that contain this keyword
df_keyword_2 = df_keyword_2.drop_duplicates()

<ipython-input-83-2415b6629a79>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_na["keyword"] = ""
<ipython-input-83-2415b6629a79>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["keyword"] = keyword
<ipython-input-83-2415b6629a79>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [84]:
# create abbreviated df by combining keyword matches and missing ctee_cds
df_abbr = pd.concat([df_na, df_keyword_1, df_keyword_2], ignore_index=True)  # add rows that contain this keyword
df_abbr = df_abbr.drop_duplicates("ein")
df_abbr

,ein,name,ico,street,city,state,zip,group,subsection,affiliation,...,sort_name,region,Description,Museum,NonMuseum,Possibly_Museum,no_ntee,unknown_ntee,general_ntee_only,keyword
0,453483149,pta oregon congress,% LEWIS ELEMENTARY PTA PRESIDENT,4401 SE EVERGREEN ST,PORTLAND,OR,97206-7763,1305,3,9,...,LEWIS ELEMENTARY PTA,3,NaN,0,0,0,1,0,0,
1,311167944,society cosmetic chemist,% SUSAN GRIFFITHS BROPHY,9378 MASON-MONTGOMERY ROAD UNIT 435,MASON,OH,45040-8827,9135,3,9,...,P&G OHIO VALLEY CHAPTER,2,NaN,0,0,0,1,0,0,
2,166364844,g p library,% M&T TR CO,285 DELAWARE AVE FL 3RD,BUFFALO,NY,14202-1885,0,3,3,...,MANUFACTURERS AND TRADERS TR CO,1,NaN,0,0,0,1,0,0,
3,237306150,open door baptist church council bluff,NaN,2701 AVENUE N,COUNCIL BLFS,IA,51501-0879,0,3,3,...,NaN,2,NaN,0,0,0,1,0,0,
4,396127477,phi upsilon omicron inc,% HUMAN ECOLOGY,HUMAN ECOLOGY 1300 LINDEN DR,MADISON,WI,53706-0000,1008,3,9,...,NU CHAPTER,2,NaN,0,0,0,1,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027228,830784635,comal county conservation alliance inc,NaN,41 STONECREST CIR,NEW BRAUNFELS,TX,78132-3200,0,3,3,...,NaN,3,Alliances & Advocacy,0,1,0,0,0,0,conserva
1027234,421386543,friend warren county conservation,NaN,15565 118TH AVE,INDIANOLA,IA,50125-8852,0,3,3,...,NaN,2,Federated Giving Programs,0,1,0,0,0,0,conserva
1027236,264490451,traveler conservation trust,NaN,3516 NE 155TH ST,LK FOREST PK,WA,98155-7412,0,3,3,...,KURT KUTAY TTEE,3,International Development,0,1,0,0,0,0,conserva
1027237,310908126,kentucky conservation committee inc,NaN,PO BOX 1152,FRANKFORT,KY,40602-1152,0,4,3,...,NaN,2,Natural Resources Conservation & Protection,0,1,0,0,0,0,conserva


In [112]:
# save frequencies to csv
df_abbr.to_csv("3_Data_IRS_ExcludeMuseumOnly_abbr.csv")

## Count Keyword Frequencies

In [85]:
matching_keywords_1 =df_keyword_1[["keyword", "name", "ntee_cd"]]
    
matching_keywords_1

,keyword,name,ntee_cd
0,hall fame,steel guitar hall fame inc,NaN
1,hall fame,philadelphia music hall fame,A23
2,hall fame,westford academy athletic hall fame corp,O50
3,hall fame,bbe hall fame,N03
4,hall fame,national softball association sport hall fame,NaN
...,...,...,...
34196,seum,brazosport museum natural science,A560
34197,seum,carthage museum historical society,A80
34198,seum,national soccer hall fame museum,N64
34199,seum,amelia island museum history inc,A540


In [86]:
matching_keywords_2 =df_keyword_2[["keyword", "name", "ntee_cd"]]
    
matching_keywords_2

,keyword,name,ntee_cd
0,infantry regiment,virginia volunteer infantry regiment,A82Z
1,infantry regiment,ohio volunteer infantry regiment band,A6C
2,infantry regiment,infantry regiment manchu association,NaN
3,infantry regiment,airborne infantry regiment association,W30
4,infantry regiment,infantry regiment association,P99
...,...,...,...
363704,conserva,coastal bend conservancy,C34
363705,conserva,vigo county conservation club inc,NaN
363706,conserva,hoosier creek conservation retriever association,NaN
363707,conserva,whitefish lake conservation organization inc,C99


In [87]:
matching_keywords_1.to_csv("3_matching_keywords_1.csv")
matching_keywords_2.to_csv("3_matching_keywords_2.csv")

### get frequencies based on keyword

In [88]:
freq_df_1 = matching_keywords_1.groupby(["keyword", "ntee_cd"]).size().reset_index(name='count')
freq_df_2 = matching_keywords_2.groupby(["keyword", "ntee_cd"]).size().reset_index(name='count')
freq_df_1 = freq_df_1.assign(Include=1)
freq_df_2 = freq_df_2.assign(Include=2)
freq_df_1_2 = freq_df_1.append(freq_df_2)

freq_df_2

,keyword,ntee_cd,count,Include
0,ancient,A05,2,2
1,ancient,A20,3,2
2,ancient,A23,16,2
3,ancient,A24Z,1,2
4,ancient,A25,1,2
...,...,...,...,...
13482,zoological,D500,68,2
13483,zoological,D50Z,3,2
13484,zoological,D99,1,2
13485,zoological,O50,1,2


In [89]:
# combine list 1 and 2
freq_df_1_2 = freq_df_1.append(freq_df_2)

In [91]:
# save frequencies to csv
freq_df_1.to_csv("3_keyword_frequencies_1s_only.csv")
freq_df_2.to_csv("3_keyword_frequencies_2s_only.csv")
freq_df_1_2.to_csv("3_keyword_frequencies_1s_2s.csv")

### get frequencies based on ctee_cd

In [101]:
# combine matches from list 1 and 2
matching_keywords = pd.concat([matching_keywords_1, matching_keywords_2], ignore_index=True)
matching_keywords.shape

(397203, 3)

In [120]:
# group by ctee_cd only
freq_df_by_ctee = matching_keywords.groupby(["ntee_cd"]).size().reset_index(name='count').sort_values("count", ascending = False)
freq_df_by_ctee

,ntee_cd,count
2741,T20,19272
303,A80,18638
2769,T22,16388
613,B82,12900
2229,P20,8377
...,...,...
1680,J034,1
1679,J033,1
1675,J022,1
1671,J00I,1


In [123]:
# save freqs by ctee_cd to csv
freq_df_by_ctee.to_csv("3_ctee_frequencies.csv", index = False)